In [ ]:
import pandas as pd 
import numpy as np
from numpy import asarray
from pathlib import Path
import cv2, pickle, csv, os, sys, re
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, MaxPooling2D, Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.layers import Dropout, Flatten

In [ ]:
# Функции для просмотра изображении
def string2image(jpeg_image_path):
    image = Image.open(jpeg_image_path)
    img_array = np.asarray(image)
    return img_array
    
def plot_cow(image):
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(string2image(image), cmap='gray')
        

In [ ]:
# Чтение таблицы ключевых точек и директории изображении. Указать путь к CSV файлу и изображениям.
keypoints_df = pd.read_csv(Path(r"C:\Users\Sony\Desktop\University\Programs\TEST\TEST_dataset.csv"))
images_dirname = Path(r"C:\Users\Sony\Desktop\University\Programs\TEST\grayscaled") 


images_ext = re.compile(".*(.jpg|.jpeg)")

filtered = list(filter(images_ext.match, os.listdir(images_dirname)))
try:
    sorted_images = sorted(filtered ,key=lambda x: int(os.path.splitext(x)[0][3:]))
except:
    try:
        sorted_images = sorted(filtered ,key=lambda x: int(os.path.splitext(x)[0][2:]))
    except:
        sorted_images = sorted(filtered)


In [ ]:
# Получение разрешения входных изображении (ширина, высота, кол.каналов)
image = Image.open(Path(os.path.join(images_dirname, sorted_images[0])))
img_array = np.asarray(image)
SHAPE = (image.size[1], image.size[0], image.layers)

In [ ]:
# Проверка. Просмотр заглавления
keypoints_df.head()

In [ ]:
# Просмотр одного изображения по номеру(индексу)         
plot_cow(Path(os.path.join(images_dirname, sorted_images[0])))

In [ ]:
# Просмотр изображении с аннотациями по номеру.
image_index = 1
keypoint_cols = list(keypoints_df.columns)[2:]
xy = keypoints_df.iloc[image_index][keypoint_cols].values.reshape((int(len(keypoints_df.columns[2:])/2), 2))
plt.plot(xy[:, 0], xy[:, 1], 'ro')
plt.imshow(string2image(Path(os.path.join(images_dirname, sorted_images[image_index]))), cmap='gray')

In [ ]:
#                            !!! ЧАСТЬ МАШИННОГО ОБУЧЕНИЯ !!!

In [ ]:
# Подготовка данных
X = np.stack([string2image(Path(os.path.join(images_dirname, image_path))) for image_path in sorted_images]).astype(np.float64)[:, :, :, np.newaxis]

In [ ]:
# Подготовка данных
y = np.vstack(keypoints_df[keypoints_df.columns[2:]].values)

In [ ]:
X.shape, X.dtype

In [ ]:
y.shape, y.dtype

In [ ]:
X_train = X / 255

In [ ]:
# Подготовка данных
output_pipe = make_pipeline(
    MinMaxScaler(feature_range=(-1, 1))
)

y_train = output_pipe.fit_transform(y)

In [ ]:
# МОДЕЛЬ МАШИННОГО ОБУЧЕНИЯ
model = Sequential()

# Входной слой
model.add(BatchNormalization(input_shape=(SHAPE)))
model.add(Conv2D(36, (5, 5), kernel_initializer='random_normal'))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Dropout(0.2))

# Второй слой
model.add(Conv2D(48, (5, 5)))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Dropout(0.2))

# Третий слой
model.add(Conv2D(64, (5, 5)))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Dropout(0.2))

# Четвертый слой
model.add(Conv2D(72, (3, 3)))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
model.add(Dropout(0.2))

# Пятый слой
model.add(Conv2D(72, (3, 3)))
model.add(Activation('sigmoid'))
model.add(Flatten())

# Шестой слой
model.add(Dense(1000, activation="sigmoid"))

# Седьмой слой
model.add(Dense(180, activation="sigmoid"))

# Восьмой слой
model.add(Dense(y.shape[1]))

In [ ]:
# Проведение машинного обучения
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.99, nesterov=True)
model.compile(optimizer=sgd, loss='mse', metrics=['accuracy'])
epochs = 11
history = model.fit(X_train, y_train, 
                 validation_split=0.2, shuffle=True, 
                 epochs=epochs, batch_size=2)

In [ ]:
# Подготовка тестовых данных
img = X_train[0, :, :, :].reshape(1, *SHAPE)
predictions = model.predict(img)

In [ ]:
# Предсказание модели
xy_predictions = output_pipe.inverse_transform(predictions).reshape(y.shape[1]//2, 2)

In [ ]:
# Проверка предсказании. Вывод данных
plt.imshow(X_train[0, :, :, 0], cmap='gray')
plt.plot(xy_predictions[:, 0], xy_predictions[:, 1], 'b*')

In [ ]:
# # Для сохранения обученной модели
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

# # Для загрузки ранее сохраненной модели
# loaded_model = pickle.load(open(filename, 'rb'))